## Calibragem

In [1]:
import numpy as np
import cv2

In [5]:
cap = cv2.VideoCapture(r'C:\Users\jv_at\tcc\Arcgis\lev_12h_1.MOV')
ret,old = cap.read()
old_gray = cv2.cvtColor(old, cv2.COLOR_BGR2GRAY)

In [6]:
fourcc = cv2.VideoWriter_fourcc(*'H264') # Cria o objeto para gravar vídeo
out = cv2.VideoWriter(r'C:\Users\jv_at\tcc\Arcgis\lev_12h_1_estabilizado.MOV', fourcc, 30.0, (old.shape[1] , old.shape[0]))

height, width = old.shape[0], old.shape[1]

In [ ]:
count = 0

while (cap.isOpened()):
    count += 1
    ret,frame = cap.read() 
        
    if ret == True:

        atual = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        pontos_chave_old = cv2.goodFeaturesToTrack(old_gray, 100, 0.0001, 10)
    
        pontos_chave_old_b = np.int0(pontos_chave_old)
        copy = frame.copy()
        
        for i in pontos_chave_old_b:
            x,y = i.ravel()
            cv2.circle(copy,(x,y),3,(0,255,255),-1)
    
        pontos_chave_atual, status, _ = cv2.calcOpticalFlowPyrLK(old_gray, atual, pontos_chave_old, np.array([]))
    
        pontos_chave_old, pontos_chave_atual = map(lambda pontos_chave_old_b: pontos_chave_old_b[status.ravel().astype(bool)], [pontos_chave_old, pontos_chave_atual])
    
    
        transform,_ = cv2.estimateAffinePartial2D(pontos_chave_old, pontos_chave_atual, True)
    
    
    
        height, width = frame.shape[0], frame.shape[1]
        
        last_transform = np.identity(3)
    
        transform = transform.dot(last_transform)
    
        transformado = cv2.warpAffine(frame, transform, (width, height))
    
    
        inverse_transform = cv2.invertAffineTransform(transform[:2])
    
        estabilizado = cv2.warpAffine(frame, inverse_transform, (width, height))
        out.write(estabilizado)
        last_transform = transform
    
        #final1 = cv2.hconcat([frame,transformado])
        #final2 = cv2.hconcat([copy, estabilizado])
        #final = cv2.vconcat([final1,final2])
        #cv2.imshow('resultado',final2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            out.write(estabilizado)  
            break
        
    else:
        break
        
cap.release()
out.release()